In [41]:
# !/home/arash/anaconda3/bin/pip install progressbar2

In [2]:
import torch
import torch.nn as nn
import torch.utils
from torchvision import datasets, transforms
import torchvision.utils
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import itertools
import time

# from progressbar import progressbar as pb

In [44]:
# FOR DEBUG PURPOSES
class Timer:
    def __init__(self, msg):
        self.msg = msg
        
    def __enter__(self):
        torch.cuda.synchronize()
        self.start = time.process_time()
        return self

    def __exit__(self, *args):
        torch.cuda.synchronize()
        self.end = time.process_time()
        self.interval = self.end - self.start
        
        print('{}: {:.5f}'.format(self.msg, self.interval))

In [45]:
manualSeed = 42
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [46]:
n_gaussian = 8  # size of latent gaussian variable
block_size = 2  # size of coordinate search blocks
n_rounds = 3  # number of coordinate search rounds 

# n_b = 0  # size of the latent binary variable
# n_o = 0  # size of the latent one-hot variable

batch_size = 64  # samples of data distribution at each iteration
latent_batch_size = np.power(32, block_size)

n_epochs = 100

In [47]:
img_size = 28  # desired resized image size
nc = 1  # number of image channels
# workers = 2  # number of workers for dataloader

In [48]:
use_cuda = torch.cuda.is_available()
gpu_ids = '2,3'
gpu_ids = [int(s) for s in gpu_ids.split(',')]
device = torch.device("cuda:{}".format(gpu_ids[0]) if use_cuda else "cpu")

# if len(gpu_ids) > 0:
#     torch.cuda.set_device(gpu_ids[0])

torch.manual_seed(42)
np.random.seed(42)

# Should the default pytorch URL may fail, edit the source code of EMNIST function and update the URL
# E.g: vim ~/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py
# Correct URL as of Jan 2020: https://cloudstor.aarnet.edu.au/plus/s/ZNmuFiuQTqZlu9W/download



dataset_mean, dataset_std = 0.1751, 0.3332
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
#                         transforms.Resize(img_size),
                        transforms.ToTensor(),
                        transforms.Normalize((dataset_mean,), (dataset_std,))
                    ])),
    batch_size=batch_size, shuffle=True, drop_last=True)


# train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=batch_size, shuffle=True, drop_last=True)

In [10]:
# dataset_mean, dataset_std = 0.1751, 0.3332
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=True, download=True,
                    transform=transforms.Compose([
#                         transforms.Resize(img_size),
                        transforms.ToTensor(),
#                         transforms.Normalize((dataset_mean,), (dataset_std,))
                    ])),
    batch_size=100000000, shuffle=True)

x = next(iter(train_loader))[0]
x.shape, x.mean(), x.std()

(torch.Size([60000, 1, 28, 28]), tensor(0.1307), tensor(0.3081))

In [49]:
class BlockCoordinateSearch():
    def __init__(self, batch_size, latent_batch_size, n_gaussian, n_rounds, block_size, criterion_no_reduction, device, netG):
        """
        Block coordinate grid search optimizer over the distribution of points 
        in the latent space.
        
        Args:
            criterion_no_reduction: loss function to compare, and MUST NOT have reduction.
        """
        
#         self.m = torch.distributions.multivariate_normal.MultivariateNormal(
#             torch.zeros(block_size), 
#             torch.eye(block_size))
        
        self.batch_size = batch_size
        self.latent_batch_size = latent_batch_size
        self.n_gaussian = n_gaussian
        self.block_size = block_size
        
        self.n_rounds = n_rounds
        
#         self.netG = netG
        self.criterion_no_reduction = criterion_no_reduction
        
#         if device.type == 'cuda':
#             torch.cuda.set_device(device)
#             self.torch = torch.cuda
#         else:
#             self.torch = torch
        self.device = device
        self.netG = netG
        
    def _sample_block(self, old_z, block_idx):
        """
        Takes the best codes and perturbs
        
        Args:
            old_z: batch_size x n_gaussian
        Returns:
            new_z: batch_size x latent_batch_size x n_gaussian
        """
        
#         gaussian_samples = torch.randn(self.batch_size, self.latent_batch_size, self.block_size, device=device)
        
        new_z = old_z.unsqueeze(1).repeat(1, self.latent_batch_size, 1)
        new_z[:, :, block_idx * self.block_size:(block_idx + 1) * self.block_size].normal_()
        
        return new_z
        
#     def _row_wise_repeat(mat, n_rep):
#         """
#         pytorch equivalent of numpy.repeat
#         """
#         n_rows = mat.shape[0]
#         return mat[np.arange(n_rows).repeat(n_rep), :]  # WARNING: np.repeat has a different functionality from torch.repeat

    def optimize(self, real):
        """
        Find the loss between the optimal fake data and the real data.
        
        Args:
            real: batch_size x dim_1 x ... x dim_k
            
        Returns:
            best_z: batch_size x n_gaussian
        """
        for round_idx, block_idx in itertools.product(range(self.n_rounds), range(n_gaussian // block_size)):
            if round_idx == block_idx == 0:
                best_z = torch.zeros(self.batch_size, self.n_gaussian, device=self.device)
#             # batch_size x latent_batch_size x n_gaussian
            new_z = self._sample_block(best_z, block_idx)

            if True:  # if netG is a deconv network
                # (batch_size * latent_batch_size) x n_gaussian x 1 x 1
                new_z_r = new_z.reshape(self.batch_size * self.latent_batch_size, self.n_gaussian, 1, 1)
            else:
                new_z_r = new_z.reshape(self.batch_size * self.latent_batch_size, self.n_gaussian)

            # (batch_size * latent_batch_size) x dim_1 x ... x dim_k
            torch.cuda.synchronize()
            with torch.no_grad():  # no need to store the gradients while searching
                fake_all = self.netG(new_z_r)

            # batch_size x latent_batch_size x dim_1 x ... x dim_k

            all_shape = [self.batch_size, self.latent_batch_size, *real.shape[1:]]

            # batch_size x latent_batch_size x dim_1 x ... x dim_k
            fake_all = fake_all.reshape(all_shape)
            real_all = real.unsqueeze(1).expand(all_shape)

            # batch_size x latent_batch_size x dim_1 x ... x dim_k
            loss = self.criterion_no_reduction(real_all, fake_all)

            # batch_size x latent_batch_size x -1
            loss = loss.reshape([*all_shape[:2], -1])

            # batch_size x latent_batch_size
            loss = loss.mean(dim=2)

            # batch_size
            _, argmin = loss.min(dim=1)

            # new_z: batch_size x latent_batch_size x n_gaussian
            # best_idx: batch_size x 1 x n_gaussian
            best_idx = argmin[:, None, None].repeat(1, 1, self.n_gaussian)

            # batch_size x 1 x n_gaussian
            best_z = torch.gather(new_z, 1, best_idx)

            # batch_size x n_gaussian
            best_z = best_z.squeeze()
                                            
#         x_train = latent_code_dist.sample(latent_batch_size).to(device)
#         y_train = real.repeat(latent_batch_size, 1)
        
#         y_pred = row_wise_repeat(netG(x_train), batch_size)

#         loss_all_modes = loss_fn(y_pred, y_train)
        

#         # Zero the gradients before running the backward pass.
#         optimizer.zero_grad()
        
#         selective_loss = loss_all_modes.mean(dim=1).reshape(latent_batch_size, batch_size).min(dim=0)[0].sum()
  

        return best_z

In [50]:
# # D_in is input dimension;
# # H is hidden dimension;
# # D_out is output dimension.

# D_in, H, D_out = n_gaussian, 100, 28 * 28

# netG = torch.nn.Sequential(
#     torch.nn.Linear(D_in, H),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H, H),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H, H),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H, D_out),
# ).to(device)

In [51]:
ngf = 64

In [52]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, gpu_ids):
        super(Generator, self).__init__()
        self.gpu_ids = gpu_ids
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( n_gaussian, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 8 x 8
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 16 x 16
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 32 x 32
#             nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
#             nn.Tanh()
#             # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        y = self.main(input)
        for dim in (2,3):
            y = y.narrow(dim, start=2, length=28)
        return y
    
# Create the generator
netG = Generator(gpu_ids).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (len(gpu_ids) > 1):
    netG = nn.DataParallel(netG, gpu_ids)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# # Handle multi-gpu if desired
# if (device.type == 'cuda') and (ngpu > 1):
#     netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

DataParallel(
  (module): Generator(
    (main): Sequential(
      (0): ConvTranspose2d(8, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
      (9): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (10): Tanh()
    )
  )
)

In [53]:
optimizer = optim.Adam(netG.parameters(), betas=(0.5, 0.999))

criterion = torch.nn.L1Loss()
criterion_no_reduction = torch.nn.L1Loss(reduction='none')

bcs = BlockCoordinateSearch(batch_size, latent_batch_size, n_gaussian, n_rounds, block_size, criterion_no_reduction, device, netG)

loss_log = []

In [54]:
fixed_noise = torch.randn(64, n_gaussian, 1, 1, device=device)
img_list = []

In [55]:
for epoch in range(1, n_epochs + 1):
    current_epoch_losses = []
    
    for iters, (data, _) in enumerate(train_loader):            
        with Timer('Iteration %d'%iters):
            # batch_size x dim_1 x ... x dim_k
            real = data.to(device)

            torch.backends.cudnn.benchmark=True  # temporary speed-up
            # batch_size x n_gaussian
            best_z = bcs.optimize(real)
            torch.backends.cudnn.benchmark=False


            if True:
                best_z = best_z.reshape(batch_size, n_gaussian, 1, 1)

            # batch_size x dim_1 x ... x dim_k
            fake = netG(best_z)

    #         x_train = latent_code_dist.sample(latent_batch_size).to(device)
    #         y_train = real.repeat(latent_batch_size, 1)

    #         y_pred = row_wise_repeat(netG(x_train), batch_size)

    #         loss_all_modes = loss_fn(y_pred, y_train)

            # in case we are predicting images with a fully connected net, we have to give it appropriate width and height!
            fake = fake.reshape(real.shape)
            loss = criterion(real, fake)
            # Zero the gradients before running the backward pass.
            optimizer.zero_grad()
    #         selective_loss = loss_all_modes.mean(dim=1).reshape(latent_batch_size, batch_size).min(dim=0)[0].sum()
            loss.backward()        

            current_epoch_losses.append(loss)

            optimizer.step()

            if iters % 50 == 0:           
                with torch.no_grad():
                    fake = netG(fixed_noise).detach().cpu()
                img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

                plt.imshow(img_list[-1].numpy().transpose(1,2,0))
                plt.show()
    
    loss_log.append(torch.tensor(current_epoch_losses).mean().detach().numpy())
    
    if True:  # if epoch % 10 == 0:
        print('Epoch: {} / Loss: {:.2f}'.format(epoch, loss_log[-1]) + '\n' + '=' * 22 + '\n')

Iteration 0: 6.45562


RuntimeError: CUDA error: out of memory (malloc at /pytorch/c10/cuda/CUDACachingAllocator.cpp:260)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x33 (0x7f374072d193 in /home/arash/.local/lib/python3.7/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x1ba1a (0x7f374096ea1a in /home/arash/.local/lib/python3.7/site-packages/torch/lib/libc10_cuda.so)
frame #2: <unknown function> + 0x1cd5e (0x7f374096fd5e in /home/arash/.local/lib/python3.7/site-packages/torch/lib/libc10_cuda.so)
frame #3: THCStorage_resize + 0xa3 (0x7f36ee5b16f3 in /home/arash/.local/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #4: at::native::empty_strided_cuda(c10::ArrayRef<long>, c10::ArrayRef<long>, c10::TensorOptions const&) + 0x636 (0x7f36efb7f856 in /home/arash/.local/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #5: <unknown function> + 0x45bcd2a (0x7f36ee4c2d2a in /home/arash/.local/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #6: <unkn

In [ ]:
plt.figure()
plt.title('Loss vs Epochs Passed')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(range(len(loss_log)), loss_log)
plt.show()

In [ ]:
grid_width = 8

x1 = np.linspace(0.02, 0.98, grid_width)
x2 = np.linspace(0.02, 0.98, grid_width)
x1, x2 = norm.ppf(x1), norm.ppf(x2)  # apply inverse normal cdf to distribute codes equally across the distribution

xv1, xv2 = np.meshgrid(x1, x2)
xv1, xv2 = xv1.reshape(-1, 1), xv2.reshape(-1, 1)

data = next(iter(train_loader))[0]
y_pred = data.cpu()
y_pred = y_pred

img = torchvision.utils.make_grid(y_pred, nrow=grid_width, padding=2, normalize=True, range=None, scale_each=False, pad_value=0)

plt.imshow(np.transpose(img, (1,2,0)))
plt.axis('off') 

plt.show()

In [ ]:
from scipy.stats import norm
import cv2 as cv

grid_width = 8

if True:
    x_test = torch.randn(grid_width * grid_width, n_gaussian, 1, 1, device=device)
else:
    x_test = torch.randn(grid_width * grid_width, n_gaussian, device=device)

y_pred = netG(x_test).detach().cpu().reshape(-1, 1, img_size, img_size)
# y_pred = y_pred.transpose(3,2)

img = torchvision.utils.make_grid(y_pred, nrow=grid_width, padding=2, normalize=False, range=None, scale_each=False, pad_value=0)

plt.figure()
plt.imshow(np.transpose(img, (1,2,0)))
plt.axis('off') 
                            
plt.show()


In [ ]:
from scipy.stats import norm
import cv2 as cv

grid_width = 4
x1 = np.linspace(0.02, 0.98, grid_width)
x1 = norm.ppf(x1)

xv_list = np.meshgrid(*([x1] * 8))
xv_list = [xv.reshape(-1, 1) for xv in xv_list]

x_test = torch.from_numpy(np.hstack(xv_list)).float().to(device)

y_pred = data.cpu()
y_pred = torch.cat([netG(x_test[(128*i):(128*(i+1)),:,None,None]).detach().cpu() for i in range(len(x_test)//128)]).reshape(-1, 1, img_size, img_size)
# y_pred = y_pred.transpose(3,2)

nrow = int(np.sqrt(len(y_pred)))
img = torchvision.utils.make_grid(y_pred, nrow=nrow, padding=2, normalize=False, range=None, scale_each=False, pad_value=0)

plt.figure(figsize=(nrow, nrow), dpi=img_size)
plt.imshow(np.transpose(img, (1,2,0)))
plt.axis('off') 
                            
# plt.show()
plt.savefig('./fig.png')

# like a fedback network while searching

In [ ]:
!ls '/home/arash/Desktop/celeba/img_align_celeba/' | wc -l